In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import category_encoders as cs
from sklearn.pipeline import FeatureUnion
from sklearn.metrics import f1_score

In [3]:

OccupancyData = pd.read_csv("lantsat.csv")

w_features=OccupancyData.iloc[:,np.arange(len(OccupancyData.columns) - 1)].copy()
w_target= OccupancyData.iloc[:,len(OccupancyData.columns) - 1].copy()


In [4]:
# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames in this wise manner yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

def feature_process(features):

    w_features_num = features;

    num_pipeline = Pipeline([
            ('selector', DataFrameSelector(list(w_features_num))),
            ('imputer', Imputer(strategy="median")),
            ('std_scaler', StandardScaler(with_mean = False)),
        ])

    full_pipeline = num_pipeline
    return full_pipeline

In [5]:
full_pipeline = feature_process(w_features)

feature_prepared = pd.DataFrame(data=full_pipeline.fit_transform(w_features),index=np.arange(1,len(w_features) + 1))

train_data = feature_prepared[:int(0.8 * len(feature_prepared))]
test_data = feature_prepared[int(0.8 * len(feature_prepared)):]
train_results = w_target[:int(0.8 * len(w_target))]
test_results = w_target[int(0.8 * len(w_target)):]

In [24]:
# Decision Trees ?
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


params = {'max_leaf_nodes': list(range(10, 100)), 'min_samples_split': [2,3,4,5]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1)
grid_search_cv.fit(train_data, train_results)
print grid_search_cv.best_estimator_

T_predict = grid_search_cv.predict(test_data)

print('_________________###################____________________')
print("The prediction accuracy using the decision tree is : {:.2f}%.".format(100*accuracy_score(test_results, T_predict)))
print('******************************************************* ')
# F1 score: 1 is perfect prediction
print("The F1 score using the decision tree is : {:.2f}.".format(f1_score(test_results,T_predict,average='weighted'))) 


Fitting 3 folds for each of 360 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 744 tasks      | elapsed:    9.3s


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=51,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')
_________________###################____________________
The prediction accuracy using the decision tree is : 82.42%.
******************************************************* 
The F1 score using the decision tree is : 0.82.


[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:   13.9s finished


In [18]:
# Random Forest 
from sklearn.ensemble import RandomForestClassifier

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(RandomForestClassifier(random_state=42), params, n_jobs=-1, verbose=1)
grid_search_cv.fit(train_data, train_results)
print grid_search_cv.best_estimator_

T_predict = grid_search_cv.predict(test_data)
print('_________________###################____________________')
print("The prediction accuracy using the Random Forest is : {:.2f}%.".format(100*accuracy_score(test_results, T_predict)))
print('******************************************************* ')
# F1 score: 1 is perfect prediction
print("The F1 score using the Random Forest is : {:.2f}.".format(f1_score(test_results,T_predict,average='micro'))) 



Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:   30.1s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=89,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
_________________###################____________________
The prediction accuracy using the Random Forest is : 87.00%.
******************************************************* 
The F1 score using the Random Forest is : 0.87.


In [19]:
# Naive Bayes
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha=1, class_prior=None, fit_prior=True)
model.fit(train_data, train_results)
MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

T_predict = model.predict(test_data)
print model
print('_________________###################____________________')
print("The prediction accuracy using the Naive Bayes is : {:.2f}%.".format(100*accuracy_score(test_results, T_predict)))
print('******************************************************* ')
# F1 score: 1 is perfect prediction
print("The F1 score using the Navie Bayes is : {:.2f}.".format(f1_score(test_results,T_predict,average='micro'))) 



MultinomialNB(alpha=1, class_prior=None, fit_prior=True)
_________________###################____________________
The prediction accuracy using the Naive Bayes is : 61.83%.
******************************************************* 
The F1 score using the Navie Bayes is : 0.62.


In [20]:
# Perception 
from sklearn.linear_model import Perceptron

model = Perceptron(alpha=1)
model.fit(train_data, train_results)

T_predict = model.predict(test_data)
print model
print('_________________###################____________________')
print("The prediction accuracy using the Perceptron is : {:.2f}%.".format(100*accuracy_score(test_results, T_predict)))
print('******************************************************* ')
# F1 score: 1 is perfect prediction
print("The F1 score using the Perceptron is : {:.2f}.".format(f1_score(test_results,T_predict,average='micro'))) 



Perceptron(alpha=1, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False)
_________________###################____________________
The prediction accuracy using the Perceptron is : 52.00%.
******************************************************* 
The F1 score using the Perceptron is : 0.52.


/Users/zhuhualong/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [21]:
# Nearest Neighbour Classifer

from sklearn.neighbors import KNeighborsClassifier


params = {'leaf_size': list(range(10, 50)), 'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9]}
grid_search_cv = GridSearchCV(KNeighborsClassifier(), params, n_jobs=-1, verbose=1)
grid_search_cv.fit(train_data, train_results)
print grid_search_cv.best_estimator_


T_predict = grid_search_cv.predict(test_data)
print('_________________###################____________________')
print("The prediction accuracy using the Nearest Neighbour Classifer is : {:.2f}%.".format(100*accuracy_score(test_results, T_predict)))
print('******************************************************* ')
# F1 score: 1 is perfect prediction
print("The F1 score using the Nearest Neighbour Classifer is : {:.2f}.".format(f1_score(test_results,T_predict,average='micro'))) 



Fitting 3 folds for each of 320 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:  2.9min finished


KNeighborsClassifier(algorithm='auto', leaf_size=10, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=8, p=2,
           weights='uniform')
_________________###################____________________
The prediction accuracy using the Nearest Neighbour Classifer is : 88.33%.
******************************************************* 
The F1 score using the Nearest Neighbour Classifer is : 0.88.


In [22]:
# SVC

from sklearn.svm import SVC
from sklearn.model_selection import KFold

param_grid = [
        {'kernel': ['rbf'], 'C': [ 2**x for x in range(0,6) ]},
    ]
inner_cv = KFold(n_splits=3, shuffle=True, random_state=1)
grid_search = GridSearchCV(SVC(), param_grid, cv=inner_cv,  n_jobs=1, scoring='accuracy',verbose=3)
grid_search.fit(train_data, train_results)
clf=grid_search.best_estimator_


T_predict = grid_search.predict(test_data)
print grid_search
print('_________________###################____________________')
print("The prediction accuracy using the SVC is : {:.2f}%.".format(100*accuracy_score(test_results, T_predict)))
print('******************************************************* ')
# F1 score: 1 is perfect prediction
print("The F1 score using the SVC is : {:.2f}.".format(f1_score(test_results,T_predict,average='micro'))) 



Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] kernel=rbf, C=1 .................................................
[CV] ................... kernel=rbf, C=1, score=0.89625, total=   0.2s
[CV] kernel=rbf, C=1 .................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] ................... kernel=rbf, C=1, score=0.89125, total=   0.2s
[CV] kernel=rbf, C=1 .................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s


[CV] ............ kernel=rbf, C=1, score=0.890556597874, total=   0.2s
[CV] kernel=rbf, C=2 .................................................
[CV] .................... kernel=rbf, C=2, score=0.9025, total=   0.2s
[CV] kernel=rbf, C=2 .................................................
[CV] ................... kernel=rbf, C=2, score=0.89875, total=   0.2s
[CV] kernel=rbf, C=2 .................................................
[CV] ............ kernel=rbf, C=2, score=0.898061288305, total=   0.2s
[CV] kernel=rbf, C=4 .................................................
[CV] .................... kernel=rbf, C=4, score=0.9025, total=   0.2s
[CV] kernel=rbf, C=4 .................................................
[CV] .................. kernel=rbf, C=4, score=0.904375, total=   0.2s
[CV] kernel=rbf, C=4 .................................................
[CV] ............ kernel=rbf, C=4, score=0.905565978737, total=   0.2s
[CV] kernel=rbf, C=8 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    6.2s finished


GridSearchCV(cv=KFold(n_splits=3, random_state=1, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'C': [1, 2, 4, 8, 16, 32]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=3)
_________________###################____________________
The prediction accuracy using the SVC is : 88.08%.
******************************************************* 
The F1 score using the SVC is : 0.88.
